# Learning Hmm By hmmlearn

## 一. hmmlearn概述
   hmmlearn实现了三种HMM模型类，按照观测状态是连续状态还是离散状态，可以分为两类。GaussianHMM和GMMHMM是连续观测状态的HMM模型，而MultinomialHMM是离散观测状态的模型，也是我们在HMM原理系列篇里面使用的模型。    
　 对于MultinomialHMM的模型，使用比较简单，"startprob_"参数对应我们的隐藏状态初始分布Π, "transmat_"对应我们的状态转移矩阵A, "emissionprob_"对应我们的观测状态概率矩阵B。   
  

In [375]:
import numpy as np
from hmmlearn import hmm
import random

下面我们用一个简单的实例来描述上面抽象出的HMM模型。这是一个盒子与球的模型，例子来源于李航的《统计学习方法》。假设我们有3个盒子，每个盒子里都有红色和白色两种球，这三个盒子里球的数量分别是：

|盒子|1|2|3|
|--|--|--|--|
|红球数|5|4|7|
|白球数|5|6|3|

按照下面的方法从盒子里抽球，开始的时候，从第一个盒子抽球的概率是0.2，从第二个盒子抽球的概率是0.4，从第三个盒子抽球的概率是0.4。以这个概率抽一次球后，将球放回。然后从当前盒子转移到下一个盒子进行抽球。规则是：如果当前抽球的盒子是第一个盒子，则以0.5的概率仍然留在第一个盒子继续抽球，以0.2的概率去第二个盒子抽球，以0.3的概率去第三个盒子抽球。如果当前抽球的盒子是第二个盒子，则以0.5的概率仍然留在第二个盒子继续抽球，以0.3的概率去第一个盒子抽球，以0.2的概率去第三个盒子抽球。如果当前抽球的盒子是第三个盒子，则以0.5的概率仍然留在第三个盒子继续抽球，以0.2的概率去第一个盒子抽球，以0.3的概率去第二个盒子抽球。如此下去，直到重复三次，得到一个球的颜色的观测序列:
$$O = \{红,白, 红\}$$


In [376]:
states = ["box 1", "box 2", "box3"]
n_states = len(states)

observations = ["red", "white"]
n_observations = len(observations)

### 二.预测问题 

也称为解码问题,给定模型$λ=(A,B,Π)$和观测序列$O=\{o_1,o_2,...o_T\}$，求给定观测序列条件下，最可能出现的对应的隐藏状态序列。

In [377]:
#隐藏状态的初始概率分布
start_probability = np.array([0.2, 0.4, 0.4])

# 转移概率矩阵A
transition_probability = np.array([
  [0.5, 0.2, 0.3],
  [0.3, 0.5, 0.2],
  [0.2, 0.3, 0.5]
])

# 观测状态概率矩阵 
emission_probability = np.array([
  [0.5, 0.5],
  [0.4, 0.6],
  [0.7, 0.3]
])

In [378]:
# 配置模型参数
model = hmm.MultinomialHMM(n_components=n_states)
model.startprob_=start_probability
model.transmat_=transition_probability
model.emissionprob_=emission_probability

In [379]:
# 用维特比算法的解码
seen = np.array([[0,1,0]]).T
logprob, box = model.decode(seen, algorithm="viterbi")
print("The ball picked:", ", ".join(map(lambda x: observations[x], seen.reshape(len(seen),))))
print("The hidden box", ", ".join(map(lambda x: states[x], box)))

('The ball picked:', 'red, white, red')
('The hidden box', 'box3, box3, box3')


也可以用predict的函数,结果一样

In [100]:
box2 = model.predict(seen)
print("The ball picked:", ", ".join(map(lambda x: observations[x],seen.reshape(len(seen),))))
print("The hidden box", ", ".join(map(lambda x: states[x], box2)))

('The ball picked:', 'red, white, red')
('The hidden box', 'box3, box3, box3')


In [101]:
print model.score(seen)
print np.exp(model.score(seen))

-2.038545309915233
0.13021800000000003


要注意的是score函数返回的是以自然对数为底的对数概率值,我们在HMM问题一中手动计算的结果是未取对数的原始概率是0.13022.
$$P(O|\lambda)=P(\{红,白, 红\}|model) = 0.13022$$

### 三. 模型参数学习问题
给定观测序列$O=\{o_1,o_2,...o_T\}$，估计模型$λ=(A,B,Π)$的参数，使该模型下观测序列的条件概率$P(O|λ)$最大

In [265]:
def generate_ball_list(n):

    # 红的是0 白的是1
    box_1 = [1,0,1,0,1,0,1,0,1,0]
    box_2 = [1,1,1,1,1,0,1,0,0,0]
    box_3 = [1,0,0,1,0,0,0,0,0,1]

    boxes= ['box1', 'box2', 'box3']

    p_init = [0.2,0.4,0.4]
    p_box1 = [0.5,0.2,0.3]
    p_box2 = [0.3,0.5,0.2]
    p_box3 = [0.2,0.3,0.5]

    boxes_dict = {'box1':[box_1,p_box1],
                  'box2':[box_2,p_box2],
                  'box3':[box_3,p_box3]}

    ball_list = []
    start_box = np.random.choice(boxes,1,p = p_init)
    ball_1 = random.sample(boxes_dict.get(start_box[0])[0],1)[0]
    ball_list.append(ball_1)
    next_box = np.random.choice(boxes,1,p=boxes_dict.get(start_box[0])[1])[0]
    ball_2 = random.sample(boxes_dict.get(next_box)[0],1)[0]
    ball_list.append(ball_2)
    for i in range(n-2):
        next_box = np.random.choice(boxes,1,p=boxes_dict.get(next_box)[1])[0]
        ball_list.append(random.sample(boxes_dict.get(next_box)[0],1)[0])
    return ball_list

In [399]:
a = generate_ball_list(10)
b = generate_ball_list(10)
c = generate_ball_list(10)
d = generate_ball_list(10)
e = generate_ball_list(10)
aT = np.array([a]).T
# c = np.array([c]).T
X2 = np.array([a])

In [400]:
a = [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
a = np.array([a])

In [401]:
model2 = hmm.MultinomialHMM(n_components=n_states, startprob_prior=np.array([0.2, 0.4, 0.4]), n_iter=50)

In [402]:
model2.fit(aT)
print model2.startprob_
print model2.transmat_
print model2.emissionprob_
print model2.score(aT)

[0.49468598 0.39124571 0.11406831]
[[0.24547411 0.49504987 0.25947603]
 [0.31609957 0.36555442 0.31834601]
 [0.23643602 0.51554292 0.24802105]]
[[0.83440668 0.16559332]
 [0.30339088 0.69660912]
 [0.80838688 0.19161312]]
-6.638011148123238
